In [4]:
import pandas as pd
import time, json, os
from tqdm import tqdm
import google.generativeai as genai
from dotenv import load_dotenv

In [5]:
# Load the .env file
load_dotenv()

# Access the key
api_key = os.getenv("GEMINI_API_KEY")

# Now use it
import google.generativeai as genai
genai.configure(api_key=api_key)


In [6]:
# Setup
genai.configure(api_key= api_key)
model = genai.GenerativeModel('gemini-2.5-pro')

# prompt

In [3]:
template = """
You are an expert assistant extracting structured eligibility features from government scheme descriptions.

Your task is to extract the following features from the given eligibility text and return a valid one-line JSON.

Eligibility Text:
"{eligibility}"

Instructions:
- For each key below, return:
    - `true` if the feature is clearly mentioned.
    - `false` if the feature is clearly *not* applicable.
    - `null` if the feature is not mentioned at all.
- Do not guess or assume. Be precise.
- If age or income values are mentioned, extract them as integers.
- If not mentioned, return `null`.
- Return ONLY a compact, valid one-line JSON. No explanation, no formatting like Markdown or code blocks.

Extract the following keys:
{{
  "is_student": bool or null,
  "is_disabled": bool or null,
  "is_female": bool or null,
  "is_male": bool or null,
  "is_transgender": bool or null,
  "is_girl_child": bool or null,
  "is_sc_st": bool or null,
  "is_obc": bool or null,
  "is_bpl": bool or null,
  "is_farmer": bool or null,
  "is_unemployed": bool or null,
  "is_self_employed": bool or null,
  "is_salaried_employee": bool or null,
  "is_daily_wage_worker": bool or null,
  "is_job_seeker": bool or null,
  "is_worker": bool or null,
  "is_labour": bool or null,
  "is_school_student": bool or null,
  "is_college_student": bool or null,
  "has_graduated": bool or null,
  "is_dropout": bool or null,
  "is_rural_resident": bool or null,
  "is_urban_resident": bool or null,
  "is_migrant_worker": bool or null,
  "is_single_woman": bool or null,
  "has_large_family": bool or null,
  "belongs_to_minority": bool or null,
  "has_bank_account": bool or null,
  "no_asset_ownership": bool or null,
  "receives_pension": bool or null,
  "has_chronic_illness": bool or null,
  "is_maternal_beneficiary": bool or null,
  "is_widow": bool or null,
  "is_senior_citizen": bool or null,
  "is_orphan": bool or null,
  "is_child_of_single_parent": bool or null,
  "is_woman_headed_household": bool or null,
  "has_disabled_family_member": bool or null,
  "lives_in_slum": bool or null,
  "is_homeless": bool or null,
  "has_no_house": bool or null,
  "has_pukka_house": bool or null,
  "has_kutcha_house": bool or null,
  "is_tribal": bool or null,
  "belongs_to_fisherfolk": bool or null,
  "belongs_to_weaver_community": bool or null,
  "is_ex_serviceman": bool or null,
  "is_person_with_hiv": bool or null,
  "is_victim_of_abuse": bool or null,
  "is_first_generation_learner": bool or null,
  "has_family_member_in_govt_service": bool or null,
  "receives_existing_govt_benefits": bool or null,
  "annual_income": integer or null,
  "min_age": integer or null,
  "max_age": integer or null,
  "max_income": integer or null
}}
"""


In [4]:
df = pd.read_csv("Cleaned_schemes.csv")

In [5]:
# Ensure no missing eligibility
df = df[df["Eligibility"].notna()].reset_index(drop=True)

In [ ]:

i = 1  # batch index
chunk = chunks[i]
print(f"\n Processing batch {i+1}/{len(chunks)}...")

# Check for previous progress
output_file = f"extracted_features_batch{i+1}.csv"
fail_file = f"failed_prompts_batch{i+1}.csv"

already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)

        time.sleep(1)

# Merge & Save
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved {output_file}")

# Save failed logs
if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f" Saved failed prompts to {fail_file}")



 Processing batch 2/8...


100%|██████████| 500/500 [50:41<00:00,  6.08s/it]

 Saved extracted_features_batch2.csv


In [ ]:
# ---- Process only 1st batch ---- #
i = 2  # batch index
chunk = chunks[i]
print(f"\n Processing batch {i+1}/{len(chunks)}...")

# Check for previous progress
output_file = f"extracted_features_batch{i+1}.csv"
fail_file = f"failed_prompts_batch{i+1}.csv"

already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)

        time.sleep(1)

# Merge & Save
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved {output_file}")

# Save failed logs
if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f" Saved failed prompts to {fail_file}")



 Processing batch 3/8...


100%|██████████| 500/500 [50:38<00:00,  6.08s/it]

 Saved extracted_features_batch3.csv


# **Batches**

# Batch 1

In [9]:

i = 0  # batch index
chunk = chunks[i]
print(f"\n Processing batch {i+1}/{len(chunks)}...")

# Check for previous progress
output_file = f"extracted_features_batch{i+1}_new.csv"
fail_file = f"failed_prompts_batch{i+1}_new.csv"

already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)

        time.sleep(1)

# Merge & Save
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved {output_file}")

# Save failed logs
if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f" Saved failed prompts to {fail_file}")



 Processing batch 1/8...


100%|██████████| 500/500 [52:47<00:00,  6.33s/it]

 Saved extracted_features_batch1_new.csv


# Batch 2

In [10]:
batch_size = 1000
chunks = [df[i:i+batch_size] for i in range(0, len(df), batch_size)]

In [11]:
i = 1
chunk = chunks[i]
print(f"\n Processing batch {i+1}/{len(chunks)}...")

# Check for previous progress
output_file = f"extracted_features_batch{i+1}.csv"
fail_file = f"failed_prompts_batch{i+1}.csv"

already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)

        time.sleep(1)

# Merge & Save
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved {output_file}")

# Save failed logs
if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f" Saved failed prompts to {fail_file}")



 Processing batch 2/4...
 Resuming: 500 rows already processed.


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:38:37<00:00,  5.92s/it]


 Saved extracted_features_batch2.csv


# Batch 3

In [12]:
batch_size = 1100
chunks = [df[i:i+batch_size] for i in range(0, len(df), batch_size)]

In [13]:
i = 2
chunk = chunks[i]
print(f"\n Processing batch {i+1}/{len(chunks)}...")

# Check for previous progress
output_file = f"extracted_features_batch{i+1}_new.csv"
fail_file = f"failed_prompts_batch{i+1}_new.csv"

already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)

        time.sleep(1)

# Merge & Save
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved {output_file}")

# Save failed logs
if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f" Saved failed prompts to {fail_file}")



 Processing batch 3/4...


100%|████████████████████████████████████████████████████████████████████████████| 1100/1100 [1:46:13<00:00,  5.79s/it]


 Saved extracted_features_batch3_new.csv


# Batch 4

In [14]:
batch_size = 1100
chunks = [df[i:i+batch_size] for i in range(0, len(df), batch_size)]

In [16]:
i = 3
chunk = chunks[i]
print(f"\n Processing batch {i+1}/{len(chunks)}...")

# Check for previous progress
output_file = f"extracted_features_batch{i+1}_new.csv"
fail_file = f"failed_prompts_batch{i+1}_new.csv"

already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)

        time.sleep(1)

# Merge & Save
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved {output_file}")

# Save failed logs
if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f" Saved failed prompts to {fail_file}")



 Processing batch 4/4...


100%|████████████████████████████████████████████████████████████████████████████████| 401/401 [38:14<00:00,  5.72s/it]

 Saved extracted_features_batch4_new.csv


# Tracking and redoing missed schemes

In [17]:
# List of processed batch files
batch_files = [
    "extracted_features_batch1_new.csv",
    "extracted_features_batch2_new.csv",
    "extracted_features_batch3_new.csv",
    "extracted_features_batch4_new.csv"
]

In [18]:
# Load and combine
combined_df = pd.concat([pd.read_csv(file) for file in batch_files], ignore_index=True)

In [19]:
# Save to a new combined CSV
combined_df.to_csv("combined_extracted_features.csv", index=False)
print(" Combined file saved as combined_extracted_features.csv")

 Combined file saved as combined_extracted_features.csv


In [28]:
# Load full dataset
full_df = pd.read_csv("Cleaned_schemes.csv")

In [29]:
# Load processed dataset
processed_df = pd.read_csv("combined_extracted_features.csv")

In [30]:
print(f" Total rows in full dataset: {len(full_df)}")

 Total rows in full dataset: 3701


## Remaining unprocessed

In [31]:
# Identify which rows are left 
remaining_df = pd.merge(
    full_df, processed_df[["URL"]],  
    on="URL", how="outer", indicator=True
).query("_merge == 'left_only'").drop(columns=["_merge"])

print(f" Remaining unprocessed rows: {len(remaining_df)}")

 Remaining unprocessed rows: 700


# Batch 5

In [34]:
chunk = remaining_df 
print(f"\n Processing final batch (Batch 5)...")
# Output files
output_file = "extracted_features_batch5_new.csv"
fail_file = "failed_prompts_batch5_new.csv"

# Resume logic
already_done = []
if os.path.exists(output_file):
    existing = pd.read_csv(output_file)
    already_done = existing.index.tolist()
    print(f" Resuming: {len(already_done)} rows already processed.")

# Extraction
results = []
fail_log = []

for idx, row in tqdm(chunk.iterrows(), total=len(chunk)):
    if idx in already_done:
        continue  # Skip already processed rows

    eligibility = str(row["Eligibility"])
    retries = 3
    for attempt in range(retries):
        try:
            prompt = template.format(eligibility=eligibility)
            response = model.generate_content(prompt)
            raw_output = response.text.strip().strip("```json").strip("```")
            parsed = json.loads(raw_output)
            results.append(parsed)
            break  # success
        except Exception as e:
            if attempt == retries - 1:
                print(f" Failed at row {idx}: {e}")
                results.append({})
                fail_log.append({
                    "row_index": idx,
                    "eligibility": eligibility,
                    "error": str(e)
                })
            else:
                time.sleep(1)  # wait before retry
    time.sleep(1)  # throttle

# Save results
features_df = pd.DataFrame(results)
final_df = pd.concat([chunk.reset_index(drop=True), features_df], axis=1)
final_df.to_csv(output_file, index=False)
print(f" Saved extracted features to: {output_file}")

if fail_log:
    pd.DataFrame(fail_log).to_csv(fail_file, index=False)
    print(f"  Saved failed prompts to: {fail_file}")



 Processing final batch (Batch 5)...


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [1:19:56<00:00,  6.85s/it]


 Saved extracted features to: extracted_features_batch5_new.csv


# Cross verify the final output csv with original csv

In [35]:
# Load both CSV files
df1 = pd.read_csv("combined_extracted_features.csv")
df2 = pd.read_csv("extracted_features_batch5_new.csv")

# Concatenate them vertically (row-wise)
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save to a new CSV (optional)
combined_df.to_csv("final_gemini_extract.csv", index=False)

In [39]:
df2["URL"].duplicated().sum()

0

In [40]:
# Load the files
cleaned_df = pd.read_csv("Cleaned_schemes.csv")
extracted_df = pd.read_csv("final_gemini_extract.csv")
# Check common key
key = "URL"  

# Merge and compare
merged = pd.merge(cleaned_df, extracted_df, on=key, how="inner", suffixes=('_cleaned', '_extracted'))

print(f" Total matched schemes: {len(merged)}")
print(" Sample comparison:")
display(merged[[key, 'Eligibility_cleaned', 'Eligibility_extracted']].head())


 Total matched schemes: 3701
 Sample comparison:


,URL,Eligibility_cleaned,Eligibility_extracted
0,https://www.myscheme.gov.in/schemes/fadsp1012e,The applicant should be a resident of Kerala S...,The applicant should be a resident of Kerala S...
1,https://www.myscheme.gov.in/schemes/icmr-pdf,ICMR-PDF is open to Indian nationals only.Fres...,ICMR-PDF is open to Indian nationals only.Fres...
2,https://www.myscheme.gov.in/schemes/tkgthe,1.\tThe applicant should be a permanent reside...,1.\tThe applicant should be a permanent reside...
3,https://www.myscheme.gov.in/schemes/skerala,The applicant should be a resident of Kerala S...,The applicant should be a resident of Kerala S...
4,https://www.myscheme.gov.in/schemes/sgassobcan...,The scholarship shall be open to students belo...,The scholarship shall be open to students belo...


In [44]:
cleaned_df["URL"].is_unique

True

In [45]:
extracted_df["URL"].is_unique

True

### Merged all 5 extracted batches (batch1 to batch5)

- Compared the merged data with cleaned_schemes.csv (via the URL column)

- Verified that all 3701 schemes matched perfectly

- extraction is 100% complete 